In [3]:
# 181115 indel_filter script written by J.Youk
# version 1.0 was completed at 15th Nov. 2018
# min_base_quality = 15
# localspan = 20
# args order = input_file, t_bam, n_bam
# if you find error, please send an e-mail to jhyouk@gmail.com

import sys,pysam
from collections import Counter

def most_common(lst):
    data = Counter(lst)
    return max(lst, key=data.get)

def pileup_bam(input_line,bam_file):
    input_split = input_line.split('\t')
    input_chr = input_split[0]
    input_pos = long(input_split[1])
    input_ref = input_split[3]
    input_alt = input_split[4][0:1]
    i=0
    nref=0
    nalt=0
    for pileupcolumn in bam_file.pileup(input_chr,input_pos-1,input_pos,min_mapping_quality=0,min_base_quality=10):
        #if pileupcolumn.pos == input_pos-1:
            #print pileupcolumn.get_query_qualities()
        for pileupread in pileupcolumn.pileups:
            if pileupcolumn.pos == input_pos-1:
                if pileupread.query_position == None:
                    continue
                else:
                    if pileupread.alignment.query_sequence[pileupread.query_position] == input_ref:
                        nref+=1
                    elif pileupread.alignment.query_sequence[pileupread.query_position] == input_alt:
                        nalt+=1
    return [str(nalt),str(nref),str(round(float(nalt)/float(nref+nalt),2))]

def snp_input_rel_pos_fc(cigar_list, abs_pos, rel_pos, input_pos):
    target = ''
    remain_distance = input_pos - abs_pos
    ya=0
    for cigar in cigar_list:
        ya += 1
        if cigar[0] == 0:
            if (abs_pos + cigar[1]) < input_pos:
                abs_pos += cigar[1]
                rel_pos += cigar[1]
                remain_distance -= cigar[1]
            elif (abs_pos + cigar[1]) == input_pos:
                abs_pos += cigar[1]
                rel_pos += cigar[1]
                remain_distance -= cigar[1]
                break
            else:
                rel_pos += remain_distance
                abs_pos += remain_distance
                remain_distance = 0
                break
        elif cigar[0] == 1:                        
            if abs_pos < input_pos:
                rel_pos += cigar[1] # no change of abs_pos and remain_distance  
            else:
                print 'unexpected result of snp_input_rel_pos_fc_cigar[0]==1'
                print input_pos
                break                           
        elif cigar[0] == 2:                                           
            if (abs_pos + cigar[1]) < input_pos:
                abs_pos += cigar[1] # no change of rel_pos 
                remain_distance -= cigar[1]
            else:
                abs_pos += remain_distance 
                rel_pos = 'Null' # there is no real rel.position of input_pos
                remain_distance = 0
                break                                
        elif cigar[0] == 3:
            if (abs_pos + cigar[1]) < input_pos:
                abs_pos += cigar[1]
                rel_pos += cigar[1]
                remain_distance -= cigar[1]                         
            else:
                rel_pos += remain_distance
                abs_pos += remain_distance
                remain_distance = 0
                break
        elif cigar[0] == 4 or cigar[0] == 5:
            if ya == 1:
                continue
            else:
                rel_pos += remain_distance
                abs_pos += remain_distance
                remain_distance = 0                    
                break
        else:
            print 'cigar[0]>5, unexpected'
            print read
            break                
    return [abs_pos, rel_pos]

def snp_ins_rel_pos_fc(cigar_list, ins_abs_pos, ins_rel_pos, input_abs_pos):
    info = []
    yb = 0
    max_yb = len(cigar_list)
    
    temp_abs_pos = 999
    temp_rel_pos = 999
    temp_insertion_length = 0
    temp_region_length = 999  #ins_abs_pos - input_abs_pos
    region_length = 999
    
    #print cigar_list
    for cigar in cigar_list:
        yb+=1
        if cigar[0] == 0 or cigar[0] == 3:
            ins_abs_pos += cigar[1]
            ins_rel_pos += cigar[1]
        elif cigar[0] == 1:
            region_length = ins_abs_pos - input_abs_pos
            
            if abs(region_length) < temp_region_length:
                temp_region_length = region_length
                temp_abs_pos = ins_abs_pos
                temp_rel_pos = ins_rel_pos
                temp_insertion_length = cigar[1]
            else:
                'blank'
            #print region_length,ins_abs_pos,ins_rel_pos
            
            ins_rel_pos += cigar[1]                                  
        elif cigar[0] == 2:
            ins_abs_pos += cigar[1] # no change of rel_pos
        elif cigar[0] == 4 or cigar[0] == 5:
            if yb == 1:
                continue
            else:            
                break
        else:
            print 'cigar[0]>5, unexpected in finding insertion1'
            print input_line
            print read
            sys.exit(1)
            
    if yb < max_yb:
        print 'unexpected yb'
        print input_abs_pos
        sys.exit(1)        
    else:
        'blank'
        
    #print temp_region_length,temp_abs_pos,temp_rel_pos,temp_insertion_length
    
    return [temp_region_length,temp_abs_pos,temp_rel_pos,temp_insertion_length]

#def lt_rt_span():
    
    
def snp_bam(input_line,tbam_file,nbam_file):
    input_split = input_line.split('\t')
    input_chr = input_split[0]
    input_pos = long(input_split[1])
    input_ref = input_split[3]
    input_alt = input_split[4][0:1]
    
    min_base_qual = 15
    local_span = 20
    
    variant_list = []
    nonvariant_list = []
    normal_list = []
    
    
    t_var=0;t_ref=0;t_other=0;t_dup=0;n_var=0;n_ref=0;n_other=0;n_dup=0
    t_initial_var_forward=0;t_initial_var_reverse=0
    var_pos_min = 151;var_pos_max=0;t_var_NM_min = 30
    
    variant_local_sequence=[]
    final_local_sequence=[]
    for i in range(41):
        variant_local_sequence.append([])
        final_local_sequence.append('')
    
    t_nonvariant_sequence = 0
    n_sequence = 0
    consensus_seq = ''
    
    whether_ins = 0
    insertion_length = 0
    region_length = 0
    input_line_ins = []
    
    
    
    for read in nbam_file.fetch(input_chr,input_pos-1,input_pos):
        if read.is_unmapped == True or read.is_duplicate == True:
            continue
        elif read.cigartuples == None:
            continue
        else:          
            
            whether_ins = 0
            insertion_length = 0
            region_length = 0
    
            ins_abs_pos = read.reference_start # include clip
            ins_rel_pos = read.query_alignment_start # consider left softclip, hardclip (ex> cigar: 10S141M => 10)
            
            input_pos_list = [] #[absolute position of input_pos, relative position of input_pos]
            cigar_list = read.cigartuples
            
            input_pos_list = snp_input_rel_pos_fc(cigar_list, read.reference_start, read.query_alignment_start, input_pos)
            input_abs_pos = input_pos_list[0]
            input_rel_pos = input_pos_list[1]
            
            if input_rel_pos == 'Null':
                n_other += 1
                continue
            q_temp = read.query_qualities[input_rel_pos - 1:input_rel_pos]
            
            if q_temp[0] < min_base_qual:
                n_other +=1            
            elif read.query_sequence[input_rel_pos - 1:input_rel_pos] == input_alt:
                n_var += 1
            else:
                n_ref += 1
                
                for cigar in cigar_list:
                    if cigar[0] == 1:
                        whether_ins += 1
                    else:
                        'blank'
                    
                if whether_ins == 0:
                    'blank'
                else:
                    nbam_ins_info = snp_ins_rel_pos_fc(cigar_list, ins_abs_pos, ins_rel_pos, input_abs_pos)
                    
                    region_length = nbam_ins_info[0]
                    ins_abs_pos = nbam_ins_info[1]
                    ins_rel_pos = nbam_ins_info[2]
                    insertion_length = nbam_ins_info[3]
                    
                    if region_length < 0:
                        temp_sequence = read.query_sequence[ins_rel_pos:ins_rel_pos-region_length] #region_length
                    else:
                        temp_sequence = read.query_sequence[ins_rel_pos + insertion_length - region_length - 1:ins_rel_pos + insertion_length] #region_length + 1 

                    temp_normal = [region_length,temp_sequence]
                    normal_list.append(temp_normal)  
  
    for read in tbam_file.fetch(input_chr,input_pos-1,input_pos):  # find nonvariant read in tumor_bam    
        temp_var_NM_min = 30
        if read.is_unmapped == True or read.is_duplicate == True:
            continue
        elif read.cigartuples == None:
            continue
        else:                    
            whether_ins = 0
            insertion_length = 0
            region_length = 0
            
            ins_abs_pos = read.reference_start # include clip
            ins_rel_pos = read.query_alignment_start # consider left softclip, hardclip (ex> cigar: 10S141M => 10)
            
            input_pos_list = [] #[absolute position of input_pos, relative position of input_pos]
            cigar_list = read.cigartuples
            
            input_pos_list = snp_input_rel_pos_fc(cigar_list, read.reference_start, read.query_alignment_start, input_pos)
            input_abs_pos = input_pos_list[0]
            input_rel_pos = input_pos_list[1]
            
            if input_rel_pos == 'Null':
                t_other += 1
                continue
                
            t1_temp = read.query_qualities[input_rel_pos - 1:input_rel_pos]
            if t1_temp[0] < min_base_qual:
                t_other +=1            
                
            elif read.query_sequence[input_rel_pos - 1:input_rel_pos] == input_alt:
                t_var += 1
                
                if len(bin(read.flag)[2:]) < 5 or bin(read.flag)[2:][-5] == '0': #forward read
                    t_initial_var_forward +=1
                else:
                    t_initial_var_reverse +=1    
                temp_var_NM_min = read.get_cigar_stats()[0]
                
                if t_var_NM_min > temp_var_NM_min[10]:
                    t_var_NM_min = temp_var_NM_min[10]
                else:
                    'blank'
                    
                if var_pos_min > input_rel_pos:
                    var_pos_min = input_rel_pos
                else:
                    'blank'
                if var_pos_max < input_rel_pos:
                    var_pos_max = input_rel_pos
                else:
                    'blank'                
                
                for m in range(local_span+1):
                    if read.query_sequence[input_rel_pos-1-m:input_rel_pos-m] == '':
                        break
                    else:
                        variant_local_sequence[local_span-m].append(read.query_sequence[input_rel_pos-1-m:input_rel_pos-m])
                for n in range(local_span):
                    if read.query_sequence[input_rel_pos+n:input_rel_pos+n+1] == '':
                        break
                    else:
                        variant_local_sequence[local_span+n+1].append(read.query_sequence[input_rel_pos+n:input_rel_pos+n+1])
                
                for k in range(local_span*2+1):
                    if len(variant_local_sequence[k]) > 0:
                        final_local_sequence[k] = most_common(variant_local_sequence[k])
                    else:
                        final_local_sequence[k] = ''

                
            else:
                t_ref += 1
                for cigar in cigar_list:
                    if cigar[0] == 1:
                        whether_ins += 1
                    else:
                        'blank'

                if whether_ins == 0:
                    'blank'
                else:    
                    tbam_ins_info = snp_ins_rel_pos_fc(cigar_list, ins_abs_pos, ins_rel_pos, input_abs_pos)
                    
                    region_length = tbam_ins_info[0]
                    ins_abs_pos = tbam_ins_info[1]
                    ins_rel_pos = tbam_ins_info[2]
                    insertion_length = tbam_ins_info[3]
                    
                    if region_length < 0:
                        temp_sequence = read.query_sequence[ins_rel_pos:ins_rel_pos-region_length] #region_length
                    else:
                        temp_sequence = read.query_sequence[ins_rel_pos + insertion_length - region_length - 1:ins_rel_pos + insertion_length] #region_length + 1 

                    temp_nonvariant = [region_length,temp_sequence]
                    nonvariant_list.append(temp_nonvariant)                    
 
    ni_list = []
    consensus_seq = ''.join(final_local_sequence)
    #print consensus_seq
    
    for read in tbam_file.fetch(input_chr,input_pos-1,input_pos):   # compare variant sequence vs (nonvariant & normal sequence)
        ni = 0
        ti = 0
        
        if read.is_unmapped == True or read.is_duplicate == True:
            continue
        elif read.cigartuples == None:
            continue
        else:
            cigar_list = read.cigartuples            
            input_pos_list = snp_input_rel_pos_fc(cigar_list, read.reference_start, read.query_alignment_start, input_pos)
            input_abs_pos = input_pos_list[0]
            input_rel_pos = input_pos_list[1]
            if input_rel_pos == 'Null':
                t_other += 1
                continue
                
            t2_temp = read.query_qualities[input_rel_pos - 1:input_rel_pos]
            if t2_temp[0] < min_base_qual:
                'blank'                          
            elif read.query_sequence[input_rel_pos - 1:input_rel_pos] != input_alt:
                if consensus_seq == '':
                    'blank'
                elif consensus_seq in read.query_sequence[0:151]:
                    t_nonvariant_sequence += 1
                else:
                    'blank'                  
            else:
                for normal in normal_list:
                    if ni in ni_list:
                        'blank'
                    else:                    
                        if normal[0] < 0:
                            var_sequence = read.query_sequence[input_rel_pos + normal[0] : input_rel_pos]                    
                        else:
                            var_sequence = read.query_sequence[input_rel_pos - 1 : input_rel_pos + normal[0]]

                        if var_sequence == normal[1]:
                            n_dup += 1
                            ni_list.append(ni)
                    ni += 1

                for nonvariant in nonvariant_list:
                    if nonvariant[0] < 0:
                        var_sequence = read.query_sequence[input_rel_pos + nonvariant[0] : input_rel_pos]                    
                    else:
                        var_sequence = read.query_sequence[input_rel_pos - 1 : input_rel_pos + nonvariant[0]]

                    if var_sequence == nonvariant[1]:
                        t_dup +=1
                        break
                        
    for read in nbam_file.fetch(input_chr,input_pos-1,input_pos):
        if read.is_unmapped == True or read.is_duplicate == True:
            continue
        elif read.cigartuples == None:
            continue
        else:
            if consensus_seq == '':
                'blank'
            elif consensus_seq in read.query_sequence[0:151]:
                n_sequence += 1
            else:
                'blank'  
    
    #print normal_list, len(normal_list)
    #print t_var,t_ref,t_other,t_dup,n_var,n_ref,n_other,n_dup
    #print variant_local_sequence
    
    return [str(t_var),str(t_ref),str(t_other),str(t_dup),str(n_var),str(n_ref),str(n_other), \
            str(n_dup),str(t_initial_var_forward),str(t_initial_var_reverse),str(var_pos_min),str(var_pos_max),str(t_var_NM_min), \
            str(round(float(t_var-t_dup)/float(t_ref+t_var),2)),str(t_nonvariant_sequence),str(n_sequence)]
    
#input_fn = sys.argv[1]
#input_file = file(input_fn)
#output_file = file(input_fn.replace('.vcf','_infoY.vcf'),'w')
input_file = file('S1-0Gy-1_union_2_normalpanel1_11.vcf.readinfo.readc.rasmy')
output_file = file('S1-0Gy-1_union_2_normalpanel1_11.readinfo.readc.rasmy_infoY.vcf','w')

#t_bam_fn = sys.argv[2]
#n_bam_fn = sys.argv[3]
#t_bam = pysam.AlignmentFile(t_bam_fn,'rb')
#n_bam = pysam.AlignmentFile(n_bam_fn,'rb')
t_bam = pysam.AlignmentFile('/home/users/team_projects/Radiation_signature/02_bam/S1-0Gy-1.s.md.ir.br.bam','rb')
n_bam = pysam.AlignmentFile('/home/users/team_projects/Radiation_signature/02_bam/N1spleen.s.md.ir.br.bam','rb')
ref_fasta = pysam.FastaFile("/home/users/jhyouk/99_reference/mouse/mm10/GRCm38.fa")

input_line = input_file.readline().strip()
while input_line[0:1] == '#':
    output_file.write(input_line + '\t' + 't_var;t_ref;t_other;t_dup;n_var;n_ref;n_other;n_dup;t_var_forward;t_var_reverse;var_pos_min;var_pos_max;t_var_NM_min;new_vaf;local_t_nonvariant;local_n' + '\n')
    #output_file.write(input_line + '\t' + 'initial_vaf' + '\n')
    input_line = input_file.readline().strip()

prev_chr = '0'
#j=0

while input_line:
    input_chr = input_line.split('\t')[0]
    if input_chr != prev_chr:
        print input_chr
        prev_chr = input_chr

    if input_line.split('\t')[1] != '90229634':
        input_line = input_file.readline().strip()
        continue
        
    info = ''

    #initial_vaf = pileup_bam(input_line,t_bam) # Calculate initial_vaf from bam file
    #info = initial_vaf[2]
    
    var_list = snp_bam(input_line,t_bam,n_bam)
    #print input_line
    #print var_list
    #break
    
    info = ';'.join(var_list)
    print input_line
    print info
    output_file.write(input_line + '\t' + info + '\n')
    input_line = input_file.readline().strip()
    #j+=1
    #if j == 10:
        #break
    
print 'The End'

1
2
3
3	90229634	.	C	T	.	01	.	.	.	.	.	PASS	DP=30;SOMATIC;SS=2;SSC=21;GPV=1E0;SPV=7.6628E-3	GT:GQ:DP:RD:AD:FREQ:DP4	0/0:.:20:20:0:0%:8,12,0,0	0/1:.:10:6:4:40%:0,6,3,1	300;11;290;11;5;3;1.667	9	9	60.0	60.0	44;93.0;147	3;57.0;106	2;22.22;7;77.78	6;66.67;0;0.0	1;11.11;0;0.0	0.0	2.0	25;1;3.85	.;.;40;36;9;9;0;23;1;9;9;0;0
8;7;3;5;0;21;5;15;7;1;53;148;1;0.2;0;2
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
The End
